# Movie Quotes Corpus EDA

# 1. Cleaning

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [6]:
pwd

'/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/movie-dialog-corpus'

In [5]:
ls

README.txt                     movie_lines.tsv
movie_characters_metadata.tsv  movie_titles_metadata.tsv
movie_conversations.tsv        raw_script_urls.tsv


In [8]:
conver_df = pd.read_csv('./movie_conversations.tsv',encoding='ISO-8859-2',warn_bad_lines =False,sep='\t',header=None)
lines_df = pd.read_csv('./movie_lines.tsv',sep='\t',error_bad_lines=False,warn_bad_lines =False,header=None)
characters_df = pd.read_csv('./movie_characters_metadata.tsv',sep='\t',warn_bad_lines =False,error_bad_lines=False,header=None)

In [9]:
conversations_cols = ["characterID_First", "characterID_second", "movieID", "list_of_utterances"]
lines_cols = ['lineID', 'characterID', 'movieID', 'character_name', 'text_of_utterances']
characters_metadata_cols = ['characterID', 'character_name', 'movieID', 'movieTitle', 'gender', 'position_in_credits']
titles_metadata_cols = ['movieID', 'movieTitle', 'movieYear', 'IMDB_rating', 'numIMDBvotes', 'Genres']

In [10]:
conver_df.columns=conversations_cols
conver_df.head()

,characterID_First,characterID_second,movieID,list_of_utterances
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']


In [11]:
lines_df.columns = lines_cols
lines_df.head()

,lineID,characterID,movieID,character_name,text_of_utterances
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [12]:
characters_df.columns=characters_metadata_cols
characters_df.head()

,characterID,character_name,movieID,movieTitle,gender,position_in_credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [13]:
conver_df.shape, lines_df.shape, characters_df.shape

((83097, 4), (293202, 5), (9034, 6))

In [14]:
df = pd.merge(lines_df, characters_df, how='inner', on=['characterID','movieID','character_name'],
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)
df.head()

,lineID,characterID,movieID,character_name,text_of_utterances,movieTitle,gender,position_in_credits
0,L1045,u0,m0,BIANCA,They do not!,10 things i hate about you,f,4
1,L985,u0,m0,BIANCA,I hope so.,10 things i hate about you,f,4
2,L925,u0,m0,BIANCA,Let's go.,10 things i hate about you,f,4
3,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.,10 things i hate about you,f,4
4,L869,u0,m0,BIANCA,Like my fear of wearing pastels?,10 things i hate about you,f,4


In [15]:
df.shape

(288447, 8)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288447 entries, 0 to 288446
Data columns (total 8 columns):
lineID                 288447 non-null object
characterID            288447 non-null object
movieID                288447 non-null object
character_name         288404 non-null object
text_of_utterances     288193 non-null object
movieTitle             288447 non-null object
gender                 288447 non-null object
position_in_credits    288447 non-null object
dtypes: object(8)
memory usage: 19.8+ MB


In [20]:
df.loc[df['text_of_utterances'].isnull()==True]

,lineID,characterID,movieID,character_name,text_of_utterances,movieTitle,gender,position_in_credits
5022,L239088,u1125,m74,JANOSZ,NaN,ghostbusters ii,m,8
13859,L283548,u1356,m90,BRUCE,NaN,his girl friday,m,3
19035,L303243,u1475,m100,JOE,NaN,innerspace,M,?
42890,L24609,u224,m14,SYKES,NaN,alien nation,m,1
67870,L535288,u2919,m192,KRUGE,NaN,star trek iii: the search for spock,?,?
67880,L535203,u2919,m192,KRUGE,NaN,star trek iii: the search for spock,?,?
67884,L535163,u2919,m192,KRUGE,NaN,star trek iii: the search for spock,?,?
67902,L535148,u2919,m192,KRUGE,NaN,star trek iii: the search for spock,?,?
67904,L535133,u2919,m192,KRUGE,NaN,star trek iii: the search for spock,?,?
71079,L540986,u3027,m198,DONATRA,NaN,star trek: nemesis,?,?


In [21]:
#Select only dialogue that is not null
df = df[df['text_of_utterances'].notnull()]

In [22]:
df.to_csv('merged_data.csv', index= False)

In [23]:
df = pd.read_csv('merged_data.csv')
df.head()

,lineID,characterID,movieID,character_name,text_of_utterances,movieTitle,gender,position_in_credits
0,L1045,u0,m0,BIANCA,They do not!,10 things i hate about you,f,4
1,L985,u0,m0,BIANCA,I hope so.,10 things i hate about you,f,4
2,L925,u0,m0,BIANCA,Let's go.,10 things i hate about you,f,4
3,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.,10 things i hate about you,f,4
4,L869,u0,m0,BIANCA,Like my fear of wearing pastels?,10 things i hate about you,f,4


In [24]:
df.shape

(288193, 8)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288193 entries, 0 to 288192
Data columns (total 8 columns):
lineID                 288193 non-null object
characterID            288193 non-null object
movieID                288193 non-null object
character_name         288150 non-null object
text_of_utterances     288193 non-null object
movieTitle             288193 non-null object
gender                 288193 non-null object
position_in_credits    288193 non-null object
dtypes: object(8)
memory usage: 17.6+ MB
